In [66]:
#Geo, the PH Geography Bot
import requests
from bs4 import BeautifulSoup
import pandas as pd
import spacy
import random
from spacy.pipeline import EntityRecognizer
import wikipedia
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler

#Fixing the Province to City Lookup Dict
df_conso2 = pd.read_csv('df_consov2.csv',encoding='utf-8')
counter = -1 #for removing the cities with parenthesis corresponding to (old names)
for c in df_conso2['City/Municipality']:
    counter += 1
    if c.find('(') != -1:
        ind = c.find('(')
        c = c[:ind].rstrip()
        df_conso2['City/Municipality'][counter] = c
df_conso2['City/Municipality'].replace(to_replace='-',value=' ',inplace=True, regex=True) #remove dashes from cities
df3 = df_conso2.groupby('Province')['City/Municipality'].apply(list).to_frame().reset_index()
ph_provtocitydict = dict(zip(df3['Province'],df3['City/Municipality']))
for v in ph_provtocitydict.values():
    for element in v:
        element.replace(u'\xa0', ' ')
provinceIdentify = []

#Provinces used in Getting Region: 
provinces = {
    'Cordillera Administrative Region' : ['Abra', 'Apayao', 'Benguet', 'Ifugao', 'Kalinga', 'Mountain Province', 'Baguio'],
    'Ilocos Region' : ['Ilocos Norte', 'Ilocos Sur', 'La Union', 'Pangasinan'],
    'Cagayan Valley' : ['Batanes', 'Cagayan', 'Isabela', 'Nueva Vizcaya', 'Quirino'],
    'Central Luzon' : ['Aurora', 'Bataan', 'Bulacan', 'Nueva Ecija', 'Pampanga', 'Tarlac', 'Zambales'],
    'Calabarzon' : ['Batangas', 'Cavite', 'Laguna', 'Rizal', 'Quezon'],
    'Southwestern Tagalog Region': ['Marinduque', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon'],
    'Bicol Region': ['Albay', 'Camarines Norte', 'Camarines Sur', 'Catanduanes', 'Masbate', 'Sorsogon'],
    'Western Visayas' : ['Aklan', 'Antique', 'Capiz', 'Guimaras', 'Iloilo', 'Negros Occidental'],
    'Central Visayas' : ['Bohol', 'Cebu', 'Negros Oriental', 'Siquijor'],
    'Eastern Visayas' : ['Biliran', 'Eastern Samar', 'Leyte', 'Northern Samar', 'Samar', 'Southern Leyte'],
    'Zamboanga Peninsula' : ['Zamboanga del Norte', 'Zamboanga del Sur', 'Zamboanga Sibugay'],
    'Northern Mindanao' : ['Bukidnon', 'Camiguin', 'Lanao del Norte', 'Misamis Occidental', 'Misamis Oriental'],
    'Davao Region' : ['Davao de Oro', 'Davao del Norte', 'Davao del Sur', 'Davao Occidental', 'Davao Oriental'],
    'SOCCSKSARGEN' : ['Cotabato', 'Sarangani', 'South Cotabato', 'Sultan Kudarat'],
    'Caraga' : ['Agusan del Norte', 'Agusan del Sur', 'Dinagat Islands', 'Surigao del Norte', 'Surigao del Sur'],
    'BARMM' : ['Basilan', 'Lanao del Sur', 'Maguindanao', 'Sulu', 'Tawi-Tawi']}

#Scraping Province Capital Data
website_url = requests.get('https://en.wikipedia.org/wiki/Provinces_of_the_Philippines', headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
soup = BeautifulSoup(website_url,'lxml')
my_table = soup.find('table',{'class':'wikitable sortable plainrowheaders toptextcells'})
df = pd.read_html(str(my_table))
df = pd.DataFrame(df[0])
df2 = df
dfPlaces = df.head(82)
dfPlaces.columns = dfPlaces.columns.droplevel(-1)
df2.columns = df2.columns.droplevel(0)
df_area = df2[['Unnamed: 1_level_1','Unnamed: 5_level_1']]
df_area = df_area.rename(columns={"Unnamed: 1_level_1": "Province", "Unnamed: 5_level_1": "Area"})
df_area = df_area.drop([82, 83])

def trimExcess(word):
    t = word.find('[') or word.find('\xa0†')
    if t== -1:
        return word
    else:
        word = word[0:t]
        return word
       
dfCapital = dfPlaces[['Province','Capital']]
provinceList = []
capitalList = []
for i in dfCapital['Province']:
    provinceList.append(trimExcess(i))
#get capital    
capitalList= []
for j in dfCapital['Capital']:
    if j == 'Manila\xa0†':
        j = 'Manila'
        capitalList.append(j)
    else:
        capitalList.append(trimExcess(j))
        
ph_provtocapitaldict = dict(zip(provinceList,capitalList))
ph_provtosizedict = dict(zip(df_area['Area'],df_area['Province']))
ph_provtosizedict = {k.replace(u'\xa0', ' ') : v.replace(u'\xa0', ' ') for k, v in ph_provtosizedict.items()}
provinceDirectory = ['Abra', 'Agusan del Norte', 'Agusan del Sur', 'Aklan', 'Albay', 'Antique', 'Apayao', 'Aurora',
'Basilan', 'Bataan', 'Batanes', 'Batangas', 'Benguet', 'Biliran', 'Bohol', 'Bukidnon', 'Bulacan', 'Cagayan',
'Camarines Norte', 'Camarines Sur', 'Camiguin', 'Capiz', 'Catanduanes', 'Cavite', 'Cebu', 'Cotabato', 'Davao de Oro',
'Davao del Norte', 'Davao del Sur', 'Davao Occidental', 'Davao Oriental', 'Dinagat Islands', 'Eastern Samar', 'Guimaras',
'Ifugao', 'Ilocos Norte', 'Ilocos Sur', 'Iloilo', 'Isabela', 'Kalinga', 'La Union', 'Laguna', 'Lanao del Norte',
'Lanao del Sur', 'Leyte', 'Maguindanao', 'Marinduque', 'Masbate', 'Misamis Occidental', 'Misamis Oriental', 'Metro Manila',
'Mountain Province', 'Negros Occidental', 'Negros Oriental', 'Northern Samar', 'Nueva Ecija', 'Nueva Vizcaya',
'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Pampanga', 'Pangasinan', 'Quezon', 'Quirino', 'Rizal',
'Romblon', 'Samar', 'Sarangani', 'Siquijor', 'Sorsogon', 'South Cotabato', 'Southern Leyte', 'Sultan Kudarat',
'Sulu', 'Surigao del Norte', 'Surigao del Sur', 'Tarlac', 'Tawi-Tawi', 'Zambales', 'Zamboanga del Norte', 'Zamboanga del Sur', 'Zamboanga Sibugay']
AreaIdentify = []

#Loading the nlp model with custom NER
nlp = spacy.load('en_core_web_md', disable=['ner'])
ner = EntityRecognizer(nlp.vocab)
ner.from_disk('./sprint4b/to/ner')
nlp.add_pipe(ner,"custom_ner")


def choice(update,context):
    msg = update.message.text
    if msg == '/geninfo':
        update.message.reply_text('Which PH province would you like me to tell you more about?')
        update.message.reply_text('Here are some examples of Provinces: {}'.format(random.sample(provinceDirectory,20)))
        return '1'
    elif msg == '/findregion':
        update.message.reply_text('Give me a province and I will tell you what region it is!')
        update.message.reply_text('Here are some examples of Provinces: {}'.format(random.sample(provinceDirectory,20)))
        return '2'
    elif msg == '/findcapital':
        update.message.reply_text('Give me a province and I will tell you its capital!')
        update.message.reply_text('Here are some examples of Provinces: {}'.format(random.sample(provinceDirectory,20)))
        return '3'
    elif msg == '/locateprovince':
        update.message.reply_text('Ask me about a PH city/municipality and I will tell you which province it belongs to!')
        update.message.reply_text('Here are some examples of Cities/Municipalities: {}'.format(random.sample(list(df_conso2['City/Municipality']),20)))
        return '4'
    elif msg == '/sizeofprovince':
        update.message.reply_text('Give me a province, and I will tell you its land area!')
        update.message.reply_text('Here are some examples of Provinces: {}'.format(random.sample(provinceDirectory,20)))                                
        return '5'
    elif msg == '/about':
        update.message.reply_text('Geo the PH Chatbot v1.0 (Last Updated: 10/31/2020) \n\n'
                                 'Created By: \n'
                                 'Gab, JB, Jelly, Nox \n'
                                 'Mentor: Julia \n\n'
                                 'Special Thanks: Bash, Eskwelabs and to the rest of Cohort V! :)')
        return 'CHOICE'
    else:
        update.message.reply_text('Error! Please return a valid command!')
        update.message.reply_text('/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
        return 'CHOICE'
    return 'CHOICE'

def capitalProvince(update,context):   
    msg = update.message.text
    try:
        doc = nlp(msg)
        provinceIdentify = []
        for ent in doc.ents:
            if str(ent) in ['What','what','is', 'The','the','Capital','capital','of','City','city','Province','province']:
                continue
            provinceIdentify.append(str(ent))
        print(provinceIdentify)
        provinceIdentify2 = ' '.join(provinceIdentify)
        print(provinceIdentify2)
        update.message.reply_text('Finding Capital of {}'.format(provinceIdentify2))
        answer = get_value(provinceIdentify2)
        if answer != None:
            update.message.reply_text('{} is the capital of {}.'.format(answer,provinceIdentify2))
            wiki_resp = wikipedia.page(answer)
            update.message.reply_text("{} ".format(wikipedia.summary(answer, sentences=2)))
            update.message.reply_text("If you want to know more, here’s a link: {}".format(wiki_resp.url))
        else:
            update.message.reply_text('Sorry, I cannot find the capital!')   
    except:
        update.message.reply_text('Please rephrase your question.')
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')  
    return 'CHOICE'
               
def provincetoRegion(update,context):
    msg = update.message.text
    doc = nlp(msg)
    try:
        ProvinceRegionIdentify=[]
        for ent in doc.ents:
            if str(ent) in ['How','how','What','is','what','Is','I','the','The','province','Province','located','Located','in','In','Area','area','Big','big','size','Size','find','Find']:
                continue
            ProvinceRegionIdentify.append(str(ent))
        ProvinceRegionIdentify2 = ' '.join(ProvinceRegionIdentify)
        update.message.reply_text('Finding Region of {}'.format(ProvinceRegionIdentify2))
        answer = provinceSearch(ProvinceRegionIdentify2)
        if answer != None:
            update.message.reply_text('The province of {} is found in the region of {}.'.format(ProvinceRegionIdentify2,answer))
            wiki_resp = wikipedia.page(answer)
            update.message.reply_text("{} ".format(wikipedia.summary(answer, sentences=2)))
            update.message.reply_text("If you want to know more, here’s a link: {}".format(wiki_resp.url))
        else:
            update.message.reply_text('Sorry, I cannot find the region of the province.')
    except:
        update.message.reply_text('Please rephrase your question.')
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
    return 'CHOICE'
    
def describeProvince(update,context):
    phrase = []
    phrase2 = []
    msg = update.message.text
    if msg != 'start':
        try:
            doc = nlp(msg)
            doc2 = [token for token in doc if token.pos_ in ('NOUN','PROPN') or token.text.capitalize() in ('Oriental')] #get only NOUN or PROPN .pos_ tokens and 'Oriental' which keeps being classified as ADJ by spaCy          
            doc2 = listToString(doc2) #break down the filtered list into string format with this function
            doc2 = nlp(doc2) #convert the string back to nlp token format
            for ent in doc2.ents: #bec we have to convert it back to token format to use .ents
                phrase.append(str(ent)) #add any detected entities to a list
                phrase2 = ' '.join(phrase) + ' province' #join the list items together, and add 'province' at the end for better searching
                if 'metro manila' in phrase2.lower() or 'city' in phrase2.lower() or 'municipality' in phrase2.lower(): #don't add province to search phrase if user is searching for Metro Manila, a city, or a municipality
                    phrase2 = phrase2[:-9]
            wiki_resp = wikipedia.page(phrase2)
            update.message.reply_text("Gaining information about {}...".format(wiki_resp.title))
            update.message.reply_text("{} ".format(wikipedia.summary(phrase2, sentences=5)))
            update.message.reply_text("If you want to know more, here’s a link: {}".format(wiki_resp.url))
        except:
            update.message.reply_text('Sorry, I do not know what you are referring to!')
    else:
        return 'CHOICE'
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
    return 'CHOICE'
     
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += str(ele) + ' '
    return str1

def get_key(val): 
    provincelist = []
    for key, value in ph_provtocitydict.items(): 
         if val in value:
                provincelist.append(key)
    return(provincelist)

def get_key2(val):  #for prov to size
    provincelist = []
    for key, value in ph_provtosizedict.items(): 
         if val in value:
                provincelist.append(key)
    return(provincelist)

def get_value(key):
    return ph_provtocapitaldict.get(key)

def provinceSearch(ent):
    for k in provinces:
        for v in provinces[k]:
            if ent in v:
                return k
    return None

def start(update, context):
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
    return 'CHOICE' 
    
def cancel(update,context):
    update.message.reply_text("Have a nice day!")
    return ConversationHandler.END
                              
def cityToProvince(update, context):
    msg = update.message.text
    try:
        doc = nlp(msg)
        provinceIdentify = []
        doc2 = [token for token in doc if token.pos_ in ('NOUN','PROPN') and token.text.lower() not in ('province', 'location')] #get only NOUN or PROPN .pos_ tokens and not include 'province'
        doc2 = listToString(doc2) #break down the filtered list into string format with this function
        doc2 = nlp(doc2) #convert the string back to nlp token format
        for ent in doc2.ents: #bec we have to convert it back to token format to use .ents
            if str(ent) in ('de','del','MacArthur'): 
                provinceIdentify.append(str(ent))
            else:
                provinceIdentify.append(str(ent))
                provinceIdentify[-1]=provinceIdentify[-1].capitalize() #capitalize format for the latest entry
        provinceIdentify2 = ' '.join(provinceIdentify) #merge the items in the list into one string
        update.message.reply_text('Finding Province of {}...'.format(provinceIdentify2))
        answer = get_key(provinceIdentify2)
        if answer != []:
            update.message.reply_text('{} is located in {}.'.format(provinceIdentify2,answer))
            provinceIdentify3 = provinceIdentify2 + ' Philippines'
            print(provinceIdentify3)
            wiki_resp = wikipedia.page(provinceIdentify3)
            update.message.reply_text("{} ".format(wikipedia.summary(provinceIdentify3, sentences=2)))
            update.message.reply_text("If you want to know more, here’s a link: {}".format(wiki_resp.url))
        else:
            update.message.reply_text('Sorry, I cannot find where it is located!')            
    except:
        update.message.reply_text("Sorry, I cannot find any additional resources on it. :(")
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
    return 'CHOICE' 
    
        
def sizeProvince(update,context):
    msg = update.message.text
    doc = nlp(msg)
    try:
        AreaIdentify = []
        for ent in doc.ents:
            if str(ent) in ['How','how','What','is','what','Is','I','the','The','province','Province','located','Located','in','In','Area','area','Big','big','size','Size','find','Find']:
                continue
            AreaIdentify.append(str(ent))
        AreaIdentify2 = ' '.join(AreaIdentify)
        update.message.reply_text('Finding Area of {}'.format(AreaIdentify2))
        answer = get_key2(AreaIdentify2)
        if answer != []:
            update.message.reply_text('{} has a size of {}.'.format(AreaIdentify2,answer))
            extractNum = ''
            for item in answer:
                for element in item:
                    if element != ' ':
                        if element.isdigit():
                            extractNum += element
                        elif element == '.':
                            extractNum += '.'
                    else:
                        break
            extractNum = float(extractNum)
            metroManilaComparison = round((extractNum/638.55),2) #Metro Manila = 638.55 sqkm
            update.message.reply_text('{} is about {} times as big as Metro Manila!'.format(AreaIdentify2,metroManilaComparison))           
        else:
            update.message.reply_text('Sorry, I cannot find the measurement!')   
    except:
        update.message.reply_text('Please rephrase your question.')
    update.message.reply_text("Hi! I'm Geo, the PH Geographical Bot. Choose what you'd like to know: \n" 
                              '/geninfo : Tell me About a Province \n' 
                              '/findregion : Find the Region of a Province \n' 
                              '/findcapital : Find the Capital of a Province \n' 
                              '/locateprovince : Locate the Province of a City/Municipality \n' 
                              '/sizeofprovince : Determine the Size of a Province \n\n' 
                              'Input the appropriate command to continue.')
    return 'CHOICE' 
    

def main():
    updater = Updater("INSERT TELEGRAM API TOKEN HERE", use_context=True)
    disp = updater.dispatcher
    conv_handler = ConversationHandler(entry_points = [CommandHandler('start',start)],
    states={'CHOICE':[MessageHandler(Filters.text,choice)],
            '1':[MessageHandler(Filters.text,describeProvince)],
            '2':[MessageHandler(Filters.text,provincetoRegion)],
            '3':[MessageHandler(Filters.text,capitalProvince)],
            '4':[MessageHandler(Filters.text,cityToProvince)],
            '5':[MessageHandler(Filters.text,sizeProvince)]},
        fallbacks=[CommandHandler('cancel',cancel)])
    disp.add_handler(conv_handler)                                   
    updater.start_polling()
    updater.idle()
    
if __name__ == '__main__':
    main()